# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [110]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [111]:
datafile = '../WeatherPy/cities_data_output.csv'
original_df = pd.read_csv(datafile)
del original_df["Unnamed: 0"]
original_df

,City,Lat,Lng,Cloudiness,Country,Date,Humidity,Temp max,Wind Speed
0,barentsburg,-46.6000,168.3333,100,NZ,2021-07-25,83,48.79,5.44
1,taolanaro,3.0707,172.7902,12,KI,2021-07-25,77,81.72,10.54
2,bluff,-30.9977,26.3286,1,ZA,2021-07-25,28,40.62,4.32
3,butaritari,11.2833,-15.8333,99,GW,2021-07-25,79,79.03,5.91
4,burgersdorp,-33.9180,25.5701,0,ZA,2021-07-25,58,58.10,2.30
...,...,...,...,...,...,...,...,...,...
581,skoghall,42.4308,130.6434,73,RU,2021-07-25,96,73.74,3.76
582,muzaffarabad,25.0213,37.2685,4,SA,2021-07-25,47,89.19,7.11
583,sao gabriel da cachoeira,33.9911,-6.8401,20,MA,2021-07-25,76,83.05,9.22
584,ambatolampy,39.2061,54.5906,1,TM,2021-07-25,68,84.16,13.80


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [112]:
# configure google maps
gmaps.configure(api_key = g_key)

In [113]:
# Store latitude and longitude in locations
locations = original_df[["Lat", "Lng"]]

# Fill NaN values and convert to float
humidity = original_df["Humidity"]

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=200,
                                 point_radius=5)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [114]:
#A max temperature lower than 80 degrees but higher than 70
#Wind speed less than 10 mph.
#humidity less than 50
# zero Cloudiness
# country = BR

good_weather = original_df.loc[(original_df["Humidity"]<=50)&(original_df["Temp max"]<=80)&(original_df["Temp max"]>70)&(original_df["Wind Speed"]<=10)&(original_df["Cloudiness"]==0)&(original_df["Country"]=="BR"),:]
good_weather

,City,Lat,Lng,Cloudiness,Country,Date,Humidity,Temp max,Wind Speed
140,mys shmidta,-21.1869,-43.9758,0,BR,2021-07-25,26,71.78,8.23
198,port moresby,-20.5386,-47.4008,0,BR,2021-07-25,19,79.47,5.37
465,cumaribo,-18.4733,-47.2003,0,BR,2021-07-25,29,76.48,7.90
567,kamthi,-24.3200,-46.9983,0,BR,2021-07-25,43,76.80,5.14


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [115]:
# params dictionary to update each iteration
good_weather["Hotel Name"] = ""
good_weather["Hotel Lat"] = ""
good_weather["Hotel Lng"] = ""

params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "best hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in good_weather.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params, verify = False)

    # convert to json
    name_address = name_address.json()
#     print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        good_weather.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        good_weather.loc[index, "Hotel Lat"] = name_address["results"][0]["geometry"]["location"]["lat"]
        good_weather.loc[index, "Hotel Lng"] = name_address["results"][0]["geometry"]["location"]["lng"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
good_weather

<ipython-input-115-74b25fb31719>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  good_weather["Hotel Name"] = ""
<ipython-input-115-74b25fb31719>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  good_weather["Hotel Lat"] = ""
<ipython-input-115-74b25fb31719>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexi

,City,Lat,Lng,Cloudiness,Country,Date,Humidity,Temp max,Wind Speed,Hotel Name,Hotel Lat,Hotel Lng
140,mys shmidta,-21.1869,-43.9758,0,BR,2021-07-25,26,71.78,8.23,Solara Hotel,-21.135172,-44.253184
198,port moresby,-20.5386,-47.4008,0,BR,2021-07-25,19,79.47,5.37,Hotel San Diogo,-20.531889,-47.389874
465,cumaribo,-18.4733,-47.2003,0,BR,2021-07-25,29,76.48,7.90,Diamante Rosa Palace Hotel,-18.474865,-47.188758
567,kamthi,-24.3200,-46.9983,0,BR,2021-07-25,43,76.80,5.14,Hotel Miami,-24.196235,-46.801717


In [118]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in good_weather.iterrows()]
locations = good_weather[["Hotel Lat", "Hotel Lng"]]
locations

,Hotel Lat,Hotel Lng
140,-21.135172,-44.253184
198,-20.531889,-47.389874
465,-18.474865,-47.188758
567,-24.196235,-46.801717


In [119]:
# check data type
locations.dtypes

# convert from object to float
locations["Hotel Lat"] = pd.to_numeric(locations["Hotel Lat"])
locations["Hotel Lng"] = pd.to_numeric(locations["Hotel Lng"])

# check data type
locations.dtypes

coordinate = locations.values.tolist()
coordinate

<ipython-input-119-48acd2f85196>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  locations["Hotel Lat"] = pd.to_numeric(locations["Hotel Lat"])
<ipython-input-119-48acd2f85196>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  locations["Hotel Lng"] = pd.to_numeric(locations["Hotel Lng"])


[[-21.135172, -44.253184],
 [-20.5318888, -47.3898743],
 [-18.4748646, -47.1887575],
 [-24.1962346, -46.8017172]]

In [120]:
# Customize the size of the figure
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig_v2 = gmaps.figure(layout=figure_layout)

In [127]:
# Assign the marker layer to a variable
markers = gmaps.marker_layer(coordinate, info_box_content = hotel_info)
# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))